# <center style="font-size:2.5em">Play with AirTable</center>

**Objectif :**  
- Créer un graphique avec les différents projets de l'équipe RGP

In [55]:
# usefull imports
from importlib.resources import open_text
import yaml
import os
import airtable
from pyvis.network import Network

Récupère les infos depuis AirTable

In [15]:
# get secrets
with open("secrets.yml") as f:
    secrets = yaml.safe_load(f)

In [16]:
# Connection to airtable
at = airtable.Airtable(secrets['rgp_base_id'], secrets['api_key'])

In [76]:
# tests
at.get('Projets')['records'][9]['fields'].keys()
# at.get('Projets')['records'][0]['fields']['Membres RGP']

odict_keys(['Nom', 'Client', 'Thématiques', 'Direction du projet RGP', 'Démarrage', 'Production/Expertise RGP', 'Type de financement', 'Sujet', 'Secteur', 'Localisation', 'Division pilote', 'DO CSTB impliquées', 'Developpeurs RGP', 'Autres interlocuteurs CSTB', 'Intervenants ext.', 'Briques CSTB', 'Rendu', 'Lien EQUIPE TEAMS', 'Dossier sur le Sharepoint', 'Fin', 'Pilotage RGP', 'Membres RGP', 'Status'])

In [78]:
# Liste des agents
agent_id_dict = {}
for record in at.get('Agents RGP')['records']:
    agent_id_dict[record['id']] = record['fields']['Nom']

# Liste des projets
project_links = dict()
for record in at.get('Projets')['records']:
    project_name = record['fields']['Nom']
    if 'Membres RGP' in record['fields']:
        project_links[project_name] = [agent_id_dict[agent_id] for agent_id in record['fields']['Membres RGP']]

## Génère un network diagram avec Pyviz (VisJS)

In [101]:
net = Network(height='600px', width='100%')

# ajoute les noeuds (les agents)
net.add_nodes(list(agent_id_dict.values()), color=['#15443C']*len(agent_id_dict))

# ajoute les noeuds (les projets)
net.add_nodes(list(project_links.keys()), color=['#C3443C']*len(project_links))

# ajoute les liens
net.add_edges([(project_name, agent_name) for project_name in project_links for agent_name in project_links[project_name]])

In [102]:
# os.remove('mygraph.html')
net.show('mygraph.html')